# Week 5 - representation and operators

This week will use DEAP to create EAs for two problem domains: the Travelling Salesman problem and Equal Groups problem.

Last week you wrote an EA for a *maximisation* problem that operated with a *binary representation*. This week you will:
 - write and test EAs for optimisation problems that are minimisation  problems
 - learn how to write an EA using an **integer** representation and a **permutation** representation
 - learn about the  operators that can be used with these representations, making use of the operators supplied with DEAP
 - write new mutation operators of your own
 
 
A useful guide the operators that are pre-defined in DEAP is <a href="https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.mutUniformInt">here</a>



## Task 1: Solving a TSP problem
Recall that the objective is find a circular tour around a set of cities such that total distance travelled is minimised.

We will assume a TSP problem in which the distances are symmetrical, i.e the distance from A to B is the same as that from B to A.
(This is not always the case in real problems - for example if there is a one-way system within a city)

As usual we need to set up by importing some libraries we will need


In [ ]:
#import some standard python packages that will be useful
import array
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


# import deap packages required
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

## Create a random instance of a TSP
To keep things simple, we will pre-define a distance matrix (generated at random) that specifies the distance between any pair of cities. We can just use this a look-up table when computing the total distance of any tour.

(You can experiment with creating different instances with differing numbers of cities, and different ranges for the distances as you wish)

In [ ]:
NUMBER_OF_CITIES = 100
MIN_distance = 50
MAX_distance = 2000

distances = np.zeros((NUMBER_OF_CITIES, NUMBER_OF_CITIES))
for city in range(NUMBER_OF_CITIES):
    cities = [ i for i in range(NUMBER_OF_CITIES) if not i == city ]
    for to_city in cities:
        distances[to_city][city] = \
            distances[city][to_city] = random.randint(MIN_distance, MAX_distance)


## Creating the base classes for a permutation problem
 We will follow the same model as in the previous tutorial.
 
We need to define a Fitness class: as this is a minimisation problem, need to use weight -1

Recall that for TSP it is natural to use a permutation representation.  In DEAP, we address this by defining an attribute we call "indices" that will use the numpy random.sample method that draws integers (without replacement) from a range *k*.

When we register the individual, we have to use *tools.initIterator* to create an individual (rather than tools.initRepeat as with the binary string) to ensure we create a unique permutation 

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()


## permutation setup for individual: 
##      name, random method to generate indices: range indices drawn from; how may genes
toolbox.register("indices", 
                 random.sample, 
                 range(NUMBER_OF_CITIES), 
                 NUMBER_OF_CITIES)

toolbox.register("individual", 
                 tools.initIterate, 
                 creator.Individual, toolbox.indices)



#  a population consist of a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)




### Write an Evaluation Function

So now we have created a test TSP instance and you have learned how to create an individual containing a permutation.
    
**Now you need to write an evaluation function that is passed an individual and returns the total distance travelled**.

*How?*  Initialise a variable total-distance = 0, then loop through the individual, and look up the distance between each pair of adjacent cities. Add this to the total distance.  Remember to add the distance between the last city in the individual to the first city as it's a circular tour

Remember that trailing comma when you return the fitness!

In [ ]:
#def evalTSP(individual):


### Define the Operators
- Register an crossover operator that works with permutations: for example you can use the *ordered crossover* method: *tools.cxOrdered()*
- Register a mutation operator that works with permutations: for example you can use *tools.mutShuffleIndexes(individual, indpb)*
- Register the evaluation funtion you created

In [ ]:
# Register the operators (select,crossover,mutate)

### Run the EA and evaluate it

1. Write some code to run an EA using the EASimple() algorithm from the previous lectures  (you can use your previous code or the supplied solutions as a starting point).
2. Add code to plot the fitness over time 
3. Experiment with the mutation probability, population size etc.


In [ ]:
# write your EA code here

### Write some new mutation operators

Instead of using the pre-defined shuffleIndexes() operator for mutation you will new write some new mutation operator functions. This is straightforward in DEAP - you simply write a new function then register it with the toolbox.

**First write a  Swap() mutation function**

- This should be passed an *individual* and should return a mutated version of the individual. It should select two random positions within the invidual, and swap their values
- the mutation function needs to return "individual," i.e. it needs that trailing comma again

- When you have written it, register it with the toolbox as the mutate operator. Rerun your algorithm and compare the performance of each operator. Remember you will need to do repeated runs with each operator and compare mean performance.


**Next, create an *invert* mutation function**

This should be passed an *individual*; it should select two random positions (*p1,p2*) and the reverse the sequence of values from *p1* to *p2*.   For example,  if the start sequence is 12345678 and we select the 3rd and 7th position, the returned invidual is 12**76543**8


Once you have written the new mutation operators, run the EA 10x with each and calculate the average fitness. Which mutation operator returns the best result?

## Task 2: Equal Groups Problem

As discussed in the lecture, in this problem, a set of *n* items have to be allocated into *g* groups such that the "total weight" of each group is equal (weight could be value/size/volume etc)

Assuming a metric *m* associated with each item (e.g. representing its weight) then for each group, *m* is summed over all items allocated to a group to give a  total value *M* for the group
The fitness is usually defined as the difference between the largest and smallest value of *M* after calculating *M* for all groups. When the difference is 0, then all groups are equal.

Hence this is a *minimisation* problem with an optimal value of 0.


### Creating an instance of an EG problem
To create a random instance, we need to define how many items there are, and then assign a random weight to each item.  We then need to create a variable that defines how many groups the items should be allocated to.
(Note that if a solution does not allocate any items to one of the groups, even if all other groups are equal, the solution will have poor fitness - that is, all groups should contain items).


In [ ]:
# create a random instance with 100 items, where each item has a weight in the range 1,100
# define the number of groups as 10
NBR_ITEMS = 100
NBR_GROUPS = 10
items = {}
for i in range(NBR_ITEMS):   # counts from 0 to NBR_ITEMS-1
    items[i] = (random.randint(1, 100))

## Representation

As shown in the lecture, we can represent a solution by a list of *n* integers (one for each item). Each integer is a number in the range 0 to *g* where *g* is the number of groups indicating what group the item is placed in

For example, asssume 10 items to be allocated to 3 groups.  An individual   "1 0 0 2 2 2 2 0 0 1" is interpreted as *item 0 goes in group 1, item 1 goes in group 0, item 2 goes in group 0, item 3 goes in group 2 ..... item 9 goes in group 0, item 10 goes in group 1*
   
Therefore an individual needs to be intialised as a list of integers, where each integer has a random value between 0 and the *(g-1)* where *g* is the number of groups. We can do this by defining an attribute "attr_int" which takes random values between 0 and *(g-1)*

When we register the individual, we use the tools.initRepeat again (as in the bitstring examples) to create a  random list of integers

As this is also a minimisation problem, we can use the same FitnessMin class as above

In [ ]:
# minimise the gap
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)


# the toolbox is the basic unit that contains all the evolutionary operators we need
toolbox = base.Toolbox()

# register the gene type with the toolbox, and tell it how to initialise the values (between 0 and NBR_GROUPS-1)
toolbox.register("attr_int", random.randint, 0, NBR_GROUPS-1)


# Structure initializers
#                         define 'individual' to be an individual
#                         consisting of NBR_ITEMS 'attr_int' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_int, NBR_ITEMS)



### a) Write the fitness function

Define a fitness function for the Equal Groups problem: the function needs to calculate the total weight of each of the *g* groups:
- for each item, add its weight (stored in items[i]) to the group that the individual indicates the item is placed in
- set *MAX* to the weight of the largest group
- set *MIN* to the weight of the smallest group
- return the difference max-min



In [ ]:
# def evalEqualGroups:

### b) Mutation operator (s)

DEAP has a pre-defined mutation operator *mutateUniformInt(individual, low, up, indpb)* that mutates an individual by replacing attributes, with probability *indpb*, by a integer uniformly drawn between *low* and *up* inclusively.

However, we can also define other valid operators. For instance a swap mutation could swap two items between groups.  Define a new mutation function that takes an individual, selects two random positions, and swaps the values at each position.


In [ ]:
# def mutateSwap:

### Write an EA to solve the Equal Groups problem
Set up an EA with the fitness function defined and with the pre-defined *mutateUniformInt* mutation operator.
Run the algorithm 10x and calculate the mean fitness.

Then switch the mutation operator to the swap operator you defined and repeat. Which method performs best?

### Extension: write a directed mutation operator

Often, we can use our knowledge of the problem to write specialised mutation operators.
In this case, a "sensible" mutation operation would be to randomly select an item from a group that is very heavy and move it to a lighter group. As its important to maintain an element of randomness in a EA, we can simply select a random item from a heavy group and "move" it to another random group (by just changing the group of the item selected to a new random value)

Implement this operator and compare to your previous results

## Task 3: Experiment!

For each of these EAs, there are a number of parameters that you have to choose:
- population size
- mutation rate
- crossover rate
- mutation operator

We will discuss tuning algorithms over the next few weeks: experiment wih different values.
A systemtatic way to do this is to automate the process, for example by running the EA inside a loop in which the population size (or mutation rate,.....) is incremented each time the loop is called. As you should test each value multiple times, you need a loop within a loop, for example the following pseudo-code will test 5 values of popsize from 50-250:


In [ ]:
#set popsize = 50
#For poptest in range(5):  # test 5 values of popsize
#   for repeats in range(5):
#       runEA and store best result (e.g in a 2d array with popsize and fitness as the 2 dimensions)
#   increment popsize by 50
#calculate mean value for each popsize